# 내용정리

> - 성경이란 특성상 개역개정이나 개역한글 버전을 사용하면 형태소 분석기에서 정확히 분리하기 어려울 것으로 예상해서 `쉬운성경` 버전을 사용했습니다.
> - 아래 코드는 sample로 `창세기`만 분석해놨는데
> - 아래 코드에 성경 리스트 데이터를 만들어 놔서 이 리스트를 활용해서 for문이든, 아님 원하는 성경 파트를 추출해서 해당 성경을 분석할 수 있습니다
> - `빈도분석`과 `워드클라우드` 분석만 진행했습니다
> - 워드 클라우드는 `image mask`를 이용할 수 있어서 새롭게 표현해봤습니다.

# 라이브러리

In [ ]:
# 정규표현식을 위한 라이브러리
import re
# 넘파이 & 판다스
import numpy as np
import pandas as pd
# 그래프 시각화 라이브러리
import matplotlib.pyplot as plt
# 형태소 분석기 라이브러리
from konlpy.tag import Okt
# 빈도분석, tfidf 라이브러리
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# 코사인 유사도 라이브러리
from sklearn.metrics.pairwise import cosine_similarity
# 에러 메세지 처리 라이브러리
import warnings
warnings.filterwarnings(action='ignore')

# matplotlib 그래프 속성 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['figure.figsize'] = (12,16)
plt.rcParams['font.size'] = 20

# 사용자 정의 함수

In [ ]:
# 텍스트 전처리 함수
def prepro_text(data, stop_words):
    # okt 형태소 분석기 객체 생성
    okt = Okt()
    # 텍스트 데이터 문자열로 통일
    data = str(data)
    
    # 영문, 특수기호, 숫자 제거
    text = re.sub(r'[^가-힣]', ' ', data)

    prepro_words = []
    # 품사 처리
    for word, tag in okt.pos(text):
        # josa와 suffix 품사 제외한 단어만 추가
        if tag not in ['Josa', 'Suffix']:
            prepro_words.append(word)
    
    # 한 문장으로 묶기
    result = ' '.join(prepro_words)

    return prepro_words


# 빈도분석 후 결과 출력
def count_analyze(texts, count_vec, cnt, color, title):
    
    # 빈도분석용 vectorizer 적합 시키기
    count_vec.fit(texts)
    
    # 단어 사전 정렬
    word_dict = sorted(count_vec.vocabulary_.items())
    # {index : word} 형식으로 단어사전 만들기
    idx2word = {idx:word for word, idx in word_dict}

    # 전체 텍스트를 하나로 묶기
    total_text = []
    total_text.append(' '.join(texts.values))

    # 앞에서 적합(fit) 시킨 vectorizer를 활용해서 전체 텍스트 백터화 하기
    count_matrix = count_vec.transform(total_text)

    count_word = []
    count_vector = []
    for i in range(cnt,0,-1):
        # {index:word} 사전을 활용해서 제일 빈도높은 순서대로 단어 추출하기
        count_word.append(idx2word[(-count_matrix.toarray()[0]).argsort()[i-1]])
        # 빈도높은 횟수를 순서대로 추출하기
        count_vector.append(count_matrix.toarray()[0][(-count_matrix.toarray()[0]).argsort()[i-1]])

    # 추출한 단어와 빈도 확인하기
    print(count_word)
    print(count_vector)

    # 가로 막대 그래프로 보여주기
    plt.barh(count_word, count_vector, color=color)
    plt.yticks(count_word)
    plt.title(f'{title} 빈도 분석')
    plt.show()

    return count_word, count_vector

# 데이터 로드

In [ ]:
# 파일 경로
old_path = './쉬운성경/구약'
new_path = './쉬운성경/신약'

old_testament = os.listdir(old_path)
new_testament = os.listdir(new_path)

# 신약만 이름 순으로 정렬
new_testament.sort()

In [ ]:
# 구약 성경 데이터 프레임
old_testament_df = pd.DataFrame(columns=['성경제목', '성경주소', '성경본문'])
# 각 구약 성경 title 가져오기
for title in old_testament:
    # 각 성경 path 세팅하기
    each_old = old_path + '/' + title
    # 각 성경 텍스트를 데이터 프레임으로 가져오기
    df = pd.read_table(each_old, encoding='cp949', names=['성경본문'])
    # '.txt'기준으로 분리한 후 앞부분을 성경제목으로 세팅
    df['성경제목'] = title.split()[1].strip('.txt')
    # 성경본문 텍스트를 str 문자열 타입으로 변경
    df['성경본문'] = df['성경본문'].astype(str)
    # 성경주소가 본문과 띄어쓰기(' ')로 구분돼 있어서 그 기준으로 분리
    df['성경주소'] = df['성경본문'].map(lambda x : x[ : x.find(' ')])
    # 위와 같이 띄어쓰기(' ') 기준으로 뒷 부분이 성경 본문
    df['성경본문'] = df['성경본문'].map(lambda x : x[x.find(' ')+1 :])
    # 제목, 주소, 본문만을 기본 데이터 프레임으로 세팅
    df = df[['성경제목', '성경주소', '성경본문']]
    # 위에서 만든 old_testament_df 에 붙이기
    old_testament_df = pd.concat([old_testament_df, df], axis=0, ignore_index=True)

# 신약 성경 데이터 프레임
new_testament_df = pd.DataFrame(columns=['성경제목', '성경주소', '성경본문'])
# 각 신약 성경 title 가져오기
for title in new_testament:
    # 각 성경 path 세팅하기
    each_new = new_path + '/' + title
    # 각 성경 텍스트를 데이터 프레임으로 가져오기
    df = pd.read_table(each_new, encoding='cp949', names=['성경본문'])
    # '.txt'기준으로 분리한 후 앞부분을 성경제목으로 세팅
    df['성경제목'] = title.split()[1].strip('.txt')
    # 성경본문 텍스트를 str 문자열 타입으로 변경
    df['성경본문'] = df['성경본문'].astype(str)
    # 성경주소가 본문과 띄어쓰기(' ')로 구분돼 있어서 그 기준으로 분리
    df['성경주소'] = df['성경본문'].map(lambda x : x[ : x.find(' ')])
    # 위와 같이 띄어쓰기(' ') 기준으로 뒷 부분이 성경 본문
    df['성경본문'] = df['성경본문'].map(lambda x : x[x.find(' ')+1 :])
    # 제목, 주소, 본문만을 기본 데이터 프레임으로 세팅
    df = df[['성경제목', '성경주소', '성경본문']]
    # 위에서 만든 new_testament_df 에 붙이기
    new_testament_df = pd.concat([new_testament_df, df], axis=0, ignore_index=True)

# 성경 리스트 정리

In [ ]:
# 구약 성경 리스트
old_testament_list = []
for title in old_testament:
  old_testament_list.append(title[title.find(' ') + 1:title.find('.')])

# 신양 성경 리스트
new_testament_list = []
for title in new_testament:
  new_testament_list.append(title.split()[1].strip('.txt'))

# 빈도분석

In [ ]:
# 창세기로 sample example
for title in old_testament_list[:1]:
  
    # 각 성경별 본문 시리즈로 불러오기
    each_old_testament = old_testament_df['성경본문'][old_testament_df['성경제목']== title]

    # 각 텍스트 본문별 전처리
    each_old_testament = each_old_testament.map(prepro_text)

    # 불용어 사전
    stop_words = ['하셨습니다', '그런데', '그', '그러자', '있는', '아니었고', '없이', '했습니다', '것', '내', '이',
                    '입니다', '너', '네', '저', '나', '해', '그리고']

    # CountVectorizer 이용하여 빈도분석
    cnt_vectorizer = CountVectorizer(
        stop_words = stop_words, # ==> 불용어사전 세팅
        ngram_range = (1,1),    # ==> 1word ngram
        max_features = 100,     # ==> 최대 특징
        min_df = 50,            # ==> 최소 문서 빈도 설정
    )

    # 빈도분석으로 그래프로 단어, 빈도 수 추출하기
    cntvec_word, cntvec_vec = analysis_by_CountVectorizer(each_old_testament,cnt_vectorizer, 20, 'tab:blue', '창세기')

    # 워드클라우드에 맞게 데이터 모양 바꾸기
    data_for_wordcloud = {word:vec for word, vec in zip(cntvec_word, cntvec_vec)}

# 이미지 적용한 워드클라우드

In [ ]:
# 적용할 이미지 path 
bible_img_path ='./open_book(1).png'
# mask 세팅을 위한 코드
bible_icon = Image.open(bible_img_path)
bible_mask = Image.new('RGB', bible_icon.size, (255,255,255))
bible_mask.paste(bible_icon,bible_icon)
mask = np.array(bible_mask)

# 이미지 마스크 적용한 워드클라우드
wc = WordCloud(
    font_path = 'Malgun Gothic',
    width = 400,    # ==> 가로 크기
    height = 400,   # ==> 세로 크기
    background_color = 'white', # ==> 배경 색
    max_words=100,  # ==> 최대 단어 세팅
    mask = mask     # ==> 위에서 세팅한 마스크 적용
)

# 도화지 생성
plt.figure(figsize=(10,10))
# 위에서 만든 data_for_wordcloud를 집어 넣어준다.
plt.imshow(wc.generate_from_frequencies(data_for_wordcloud).to_image())
# 축은 사용하지 않는다.
plt.axis('off')
# 이미지 출력
plt.show()